# Testing Python MINIMOD against GAMS MINIMOD using Cameroon Data

GAMS code comes from [Cameroon_maxvarobust2.gms](robustness_data/Cameroon_maxvarobust2.gms)

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import pandas as pd
import os

from minimod import MonteCarloMinimod, Minimod


Using Python-MIP package version 1.7.3


## Getting the Data Ready

In [4]:
df_costs = (
    pd.read_csv('robustness_data/cost_data.csv')
    .set_index(['intervention', 'region', 'time'])
    )

df_eff_coverage = (
    pd.read_csv("robustness_data/data_in_gms_code/processed_data/benefits_8_13_2018_maxdata.csv")
    .drop(['Unnamed: 0'], axis=1)
    .replace(to_replace={'north' : 'North', 'south' : 'South', 'cities' : 'Cities'})
    .set_index(['intervention', 'region'])
    )

df_pop = (
    pd.read_csv("robustness_data/data_in_gms_code/processed_data/Age-group_pop_2015data.csv")
    .rename({'0' : 'population'}, axis=1)
    .replace(to_replace={'north' : 'North', 'south' : 'South', 'cities' : 'Cities'})
    .set_index(['region', 'time'])    
)


In [5]:
# put benefits together with population

df_benefits = (
    df_eff_coverage
    .merge(df_pop, left_index = True, right_index = True)
    .assign(benefit_mean = lambda df: df['mean']*df['population'],
            benefit_sd = lambda df: df['sd']*df['population']
            )
    .reset_index()
    .set_index(['intervention', 'region', 'time'])
)

In [6]:

def data_revision(ben_data, 
                  intervention_to_replace,
                  intervention_new,
                  time_to_replace,
                  region_to_replace,
                  eff_cov_data = df_eff_coverage, 
                  pop_data = df_pop):
    
    # Merge population data and benefits data at specified 
    # intervention and time period
    revised_values = (
        eff_cov_data
        .loc[(intervention_new, region_to_replace), :]
        .merge(pop_data
               .loc[(region_to_replace, time_to_replace), :], 
               left_index = True, 
               right_index = True)
        .assign(new_ben_mean = lambda df: df['mean']*df['population'],
                new_ben_sd = lambda df: df['sd']*df['population'])
        [['new_ben_mean', 'new_ben_sd']]
        .reset_index()
        .set_index(['intervention', 'region', 'time'])
        .sort_index()
        .values
        )
    
    df = ben_data.copy(deep=True)
    
    # Now get data that will be replaced    
    df.loc[(intervention_to_replace, 
                  region_to_replace, 
                  time_to_replace), ['benefit_mean', 'benefit_sd']] = revised_values
    
    print(f"Done with {intervention_to_replace}")

    return df

def observation_revision(ben_data, 
                         values_to_replace,
                         intervention_to_replace,
                         region_to_replace,
                         time_to_replace,
                         pop_data = df_pop
                         ):
    
    df = ben_data.copy(deep=True)
    
    df.loc[(intervention_to_replace, region_to_replace, time_to_replace), 
           ['benefit_mean', 'benefit_sd']] = \
               values_to_replace*pop_data.loc[(region_to_replace, time_to_replace)].values
    
    print(f"Changed {intervention_to_replace} to {values_to_replace} in {region_to_replace}")
    
    return df
    
    
    


In [7]:
## Now we make all the changes that were in the GAMS file.

new_benefits_df = (
    df_benefits
    .pipe(data_revision, 
          intervention_to_replace = 'fortoil',
          intervention_new = 'dwoil',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'capoil',
          intervention_new = 'capdwoil',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'oilcube',
          intervention_new = 'capdwoil',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'capoilcube',
          intervention_new = 'capdwoilcube',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'capoilmaize',
          intervention_new = 'capdwoilmaize',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'oilmaize',
          intervention_new = 'dwoilmaize',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'oilcubemaize',
          intervention_new = 'dwoilcubemaize',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(data_revision,
          intervention_to_replace = 'capoilcubemaize',
          intervention_new = 'capdwoilcubemaize',
          time_to_replace = 1,
          region_to_replace = slice(None))
    .pipe(observation_revision,
        values_to_replace = [0.188647039,0.056720184],
        intervention_to_replace = 'fortoil',
        region_to_replace = 'South',
        time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.184657986,0.051138841],
          intervention_to_replace = 'fortoil',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.361048462,0.052177091],
          intervention_to_replace = 'fortoil',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.271910918,0.03636394],
          intervention_to_replace = 'capoil',
          region_to_replace = 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.552711992,0.043181427],
          intervention_to_replace = 'capoil',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.429750048,0.027707883],
          intervention_to_replace = 'capoil',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.317578384,0.041965529],
          intervention_to_replace = 'oilcube',
          region_to_replace = 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.452445346,0.062957324],
          intervention_to_replace = 'oilcube',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.460061855,0.034025083],
          intervention_to_replace = 'oilcube',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.335812702,0.335812702],
          intervention_to_replace = 'capoilcube',
          region_to_replace = 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.66038935,0.66038935],
          intervention_to_replace = 'capoilcube',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.481598393,0.481598393],
          intervention_to_replace = 'capoilcube',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.281736205,0.281736205],
          intervention_to_replace = 'capoilmaize',
          region_to_replace = 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.449118915,0.449118915],
          intervention_to_replace = 'capoilmaize',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.381499444,0.381499444],
          intervention_to_replace = 'capoilmaize',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.193206296,0.039506241],
          intervention_to_replace = 'oilmaize',
          region_to_replace = 'South',
          time_to_replace = 2
          )
    .pipe(observation_revision,
          values_to_replace = [0.298684737,0.044670404],
          intervention_to_replace = 'oilmaize',
          region_to_replace = 'North',
          time_to_replace = 2
          )
    .pipe(observation_revision,
          values_to_replace = [0.314661303,0.031472923],
          intervention_to_replace = 'oilmaize',
          region_to_replace = 'Cities',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.281736205,0.038565253],
          intervention_to_replace = 'oilcubemaize',
          region_to_replace = 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace= [0.449118915,0.046413903],
          intervention_to_replace = 'oilcubemaize',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.381499444,0.033349636],
          intervention_to_replace = 'oilcubemaize',
          region_to_replace = 'Cities',
          time_to_replace=2)
    .pipe(observation_revision,
          values_to_replace= [0.331342794,0.032749485],
          intervention_to_replace= 'capoilcubemaize',
          region_to_replace= 'South',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.662643196,0.040875425],
          intervention_to_replace = 'capoilcubemaize',
          region_to_replace = 'North',
          time_to_replace = 2)
    .pipe(observation_revision,
          values_to_replace = [0.471177607,0.02437671],
          intervention_to_replace = 'capoilcubemaize',
          region_to_replace = 'Cities',
          time_to_replace = 2)
)




Done with fortoil
Done with capoil
Done with oilcube
Done with capoilcube
Done with capoilmaize
Done with oilmaize
Done with oilcubemaize
Done with capoilcubemaize
Changed fortoil to [0.188647039, 0.056720184] in South
Changed fortoil to [0.184657986, 0.051138841] in North
Changed fortoil to [0.361048462, 0.052177091] in Cities
Changed capoil to [0.271910918, 0.03636394] in South
Changed capoil to [0.552711992, 0.043181427] in North
Changed capoil to [0.429750048, 0.027707883] in Cities
Changed oilcube to [0.317578384, 0.041965529] in South
Changed oilcube to [0.452445346, 0.062957324] in North
Changed oilcube to [0.460061855, 0.034025083] in Cities
Changed capoilcube to [0.335812702, 0.335812702] in South
Changed capoilcube to [0.66038935, 0.66038935] in North
Changed capoilcube to [0.481598393, 0.481598393] in Cities
Changed capoilmaize to [0.281736205, 0.281736205] in South
Changed capoilmaize to [0.449118915, 0.449118915] in North
Changed capoilmaize to [0.381499444, 0.381499444] i

In [8]:
# Now we put together with costs

df_ready = (
    new_benefits_df
    .merge(df_costs, left_index = True, right_index = True)
    .reset_index()
)



In [16]:
df_ready

,intervention,region,time,mean,sd,population,benefit_mean,benefit_sd,costs
0,fortoil,Cities,8,0.411854,0.041279,1505019,619848.134356,62125.587495,161698.55
1,capoilcube2,North,7,0.000000,NaN,1845177,0.000000,NaN,0.00
2,OilCDTI,North,1,0.667000,NaN,1476393,984754.131000,NaN,901538.31
3,oilcube,North,5,0.496486,0.063975,1712949,850455.765913,109585.127744,285162.63
4,capcube2,North,5,0.000000,NaN,1712949,0.000000,NaN,0.00
...,...,...,...,...,...,...,...,...,...
1435,CubeMaizeBCC,Cities,1,0.317000,NaN,1213402,384648.434000,NaN,2318702.90
1436,dwoilmaize,South,10,0.149137,0.038406,2567812,382957.051879,98620.504670,107706.39
1437,oilmaize,North,10,0.355625,0.045958,2063370,733785.159789,94827.927216,170511.99
1438,BCC,South,5,0.152923,NaN,2284662,349377.777188,NaN,5714743.01


## Running a non-Monte Carlo Run

Since one run of the GAMS code is a non-Monte Carlo run


In [13]:
m = Minimod(solver_type='costmin', minimum_benefit = 'capdwoil')


              MiniMod Nutrition Intervention Tool
              Optimization Method: MIN
              Version: 0.0.3dev
              Solver: CBC,
              Show Output: True
              
              


In [17]:
m.fit(data = df_ready, 
    intervention='intervention',
    space='region',
    time='time',
    benefits = 'benefit_mean',
    costs='costs',
    minimum_benefit = 'capdwoil',
    benefit_title = "Children Saved",
    all_space = ['oil', 'cube', 'maize'],
    all_time = ['cube','maize'],
    time_subset= [1,2,3],
    )

[Note]: Processing Data...
[Note]: Creating Base Model with constraints
[Note]: Optimizing...
[Note]: Optimal Solution Found


In [18]:
m.report()

+----------------------------+----------------------------+
| MiniMod Solver Results     |                            |
| Method:                    | MIN                        |
| Solver:                    | CBC                        |
| Optimization Status:       | OptimizationStatus.OPTIMAL |
| Number of Solutions Found: | 1                          |
+----------------------------+----------------------------+
+-----------------------------+-------+
| No. of Variables:           |  2880 |
| No. of Integer Variables:   |  2880 |
| No. of Constraints          | 17804 |
| No. of Non-zeros in Constr. | 40624 |
+-----------------------------+-------+
+---------------------+-------------------+
| Minimum Benefit     | capdwoil          |
| Total Cost          | 7782948.089999998 |
| TotalChildren Saved | 7782948.089999998 |
+---------------------+-------------------+
+------------------+---+
| Cost per Benefit | 1 |
+------------------+---+
+-----------------------------------+--+
| To

In [20]:
m.opt_df.loc[lambda df: df['opt_vals']>0]

opt_vals   opt_benefit  opt_costs
intervention region time                                   
oilcube      Cities 1            1  4.642441e+05  199392.54
                    2            1  5.761102e+05  199392.54
                    3            1  6.151391e+05  199392.54
                    4            1  6.338662e+05  285162.63
                    5            1  6.534168e+05  285162.63
                    6            1  6.737385e+05  285162.63
                    7            1  6.948714e+05  285162.63
                    8            1  7.168378e+05  285162.63
                    9            1  7.398120e+05  285162.63
                    10           1  7.641423e+05  285162.63
             North  1            1  7.460961e+05  199392.54
                    2            1  6.933146e+05  199392.54
                    3            1  7.896417e+05  199392.54
                    4            1  8.194859e+05  285162.63
                    5            1  8.504558e+05  285162.63
                    6            1  8.826782e+05  285162.63
                    7            1  9.161052e+05  285162.63
                    8            1  9.507793e+05  285162.63
                    9            1  9.868028e+05  285162.63
                    10           1  1.024435e+06  285162.63
             South  1            1  5.070942e+05  199392.54
                    2            1  6.768974e+05  199392.54
                    3            1  7.048543e+05  199392.54
                    4            1  7.212341e+05  285162.63
                    5            1  7.379097e+05  285162.63
                    6            1  7.551190e+05  285162.63
                    7            1  7.728088e+05  285162.63
                    8            1  7.909867e+05  285162.63
                    9            1  8.097466e+05  285162.63
                    10           1  8.293627e+05  285162.63

In [ ]:
# Run Montecarlo

a = MonteCarloMinimod(solver_type = 'costmin', 
                        data = df_ready, 
                        intervention_col='intervention',
                        space_col='region',
                        time_col='time',
                        benefit_mean_col = 'benefit_mean',
                        benefit_sd_col= 'benefit_sd',
                        cost_col='costs',
                        minimum_benefit = 'capdwoil')


In [ ]:
sim = a.fit_all_samples(N = 100,
                        all_space = ['oil', 
                                     'cube', 
                                     'maize'
                                     ],
                        all_time = [
                            'cube',
                            'maize'
                        ],
                        time_subset= [1,2,3])




In [ ]:
a.report(perc_intervention_appeared=True, 
         avg_time = True,
         avg_space = True)



In [ ]:

a.plot_opt_hist(save = "sim_results.png")




In [ ]:
a.plot_sim_trajectories(save = 'sim_traj.png')
